# 1. Query tools

This Section introduces three methods to query the ALMA archive:

- [1.1 - Query by target name (<i>alminer.target</i>)](#1.1-Query-by-target-name)<br>
- [1.2 - Query a catalog (<i>alminer.catalog</i>)](#1.2-Query-by-position)<br>
- [1.3 - Query by ALMA keywords (<i>alminer.keysearch</i>)](#1.3-Query-by-ALMA-keywords)

<h3>General notes about the querying functions:</h3>

* All querying functions search the ALMA archive for public data by default. To include both public and proprietary data in the search, set *public=None*. Similarly, to search for only propietary data, set *public=False*. 
* All querying functions search the ALMA archive for both published and unpublished data. To include only unpublished data, set *published=False*.
* The querying functions will by default print a summary of the observations, including a list of target names. For large queries, it is useful to turn this feature off in order to not have a long list of targets printed to screen. To turn off this feature, simply set *print_targets=False*.
* The queried archive service is by default the Europeran one (ESO). Other services can be specified through the *tap_service* argument. Options are: ESO, NRAO, or NAOJ.
* The queries return all possible observations in [PANDAS DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html) format that can be used to further narrow down your search as demonstrated in [Section 2](2_filter_explore.ipynb).

<h3>Load libraries </h3>

In [1]:
import alminer
import pandas
from astropy.io import ascii

## 1.1 Query by target name

The [<code>alminer.target</code>](../pages/api.rst#alminer.target) function allows one to query objects by name. This function uses the Astropy [SESAME resolver](http://cds.u-strasbg.fr/cgi-bin/Sesame) which searches multiple databases (Simbad, NED, VizieR) to obtain the coordinates of the object of interest, and then queries the ALMA archive for all observations that contain those coordinates (corresponding to the case of *point=True* which is the default). When *point=False*, the function will return all observations that overlap with a cone extending the position of interest and a search radius around it. The search radius is by default 1.0 arcminute, but can be modified using the *search_radius* keyword (in arcmin units). 

### Example 1.1.1: query two sources by name

In [2]:
myquery = alminer.target(['Orion KL', "AB Aur"])

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 23
Number of observations = 38
Number of unique subbands = 129
Total number of subbands = 160
18 target(s) with ALMA data = ['OMC1_NW', 'Orion-KL', 'orion_kl', 'BN-KL', 'OMC1_SE', 'Orion_KL_Field_3_North-west_Clump', 'BN', 'OrionKL', 'Orion', 'Orion_KL', 'orion-IRc2', 'OMC-1', 'ONC', 'Orion1', 'OMC-1_Region2', 'ONC_Mosaic', 'Orion_Source_I', 'Orion_BNKL_source_I']
--------------------------------
Target = AB Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with ALMA data = ['AB_Auriga', 'AB_Aur', 'ab_aurigae']
--------------------------------


### Example 1.1.2: query a list of objects by name

First create a catalog or a list of object names. In this example, the catalog `Sample_cat.dat` has the following content:
```
 Name     RA      DEC   
------ -------- --------
AB_Aur  73.9412  30.5511
AK_Sco 253.6867 -36.8886
AS_310 278.3383  -4.9683
AS_470 324.0592  57.3586
AS_477 328.1421  47.2289
```

Note that the column that is used is the *Name* column and the coordinates are ignored in this example.

In [3]:
mylist = ascii.read("Sample_cat.dat", header_start=0, data_start=1)
myquery = alminer.target(mylist['Name'])

alminer.target results 
Target = AB_Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with ALMA data = ['AB_Auriga', 'AB_Aur', 'ab_aurigae']
--------------------------------
Target = AK_Sco
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 12
Total number of subbands = 12
2 target(s) with ALMA data = ['AK_Sco', 'HIP_82747']
--------------------------------
Target = AS_310
--------------------------------
No observations found.
--------------------------------
Target = AS_470
--------------------------------
No observations found.
--------------------------------
Target = AS_477
--------------------------------
No observations found.
--------------------------------


### Example 1.1.3: include proprietary data

In [4]:
myquery = alminer.target(mylist['Name'], public=None)

alminer.target results 
Target = AB_Aur
--------------------------------
Number of projects = 6
Number of observations = 10
Number of unique subbands = 56
Total number of subbands = 62
3 target(s) with ALMA data = ['AB_Aur', 'AB_Auriga', 'ab_aurigae']
--------------------------------
Target = AK_Sco
--------------------------------
Number of projects = 4
Number of observations = 4
Number of unique subbands = 16
Total number of subbands = 16
2 target(s) with ALMA data = ['AK_Sco', 'HIP_82747']
--------------------------------
Target = AS_310
--------------------------------
No observations found.
--------------------------------
Target = AS_470
--------------------------------
No observations found.
--------------------------------
Target = AS_477
--------------------------------
No observations found.
--------------------------------


### Example 1.1.4: account for mosaics

The [<code>alminer.target</code>](../pages/api.rst#alminer.target) function will by default search whether any ALMA observations contain the target of interest's position. To search whether any ALMA observations overlap with a larger region of interest, one can set the argument *point=False* and provide a search radius in arcminutes using the *search_radius* argument. The search radius is by default 1.0 arcminute.

In [5]:
myquery = alminer.target(['Orion KL', "AB Aur"], point=False, search_radius=2.0)

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 37
Number of observations = 125
Number of unique subbands = 312
Total number of subbands = 576
57 target(s) with ALMA data = ['OrionKL', 'Orion H2O maser outburst', 'OrionField1-1', 'OrionField2', 'OrionField1-2', 'f1', 'f7', 'f5', 'f8', 'f4', 'f3', 'orion_kl', 'Orion_Source_I', 'BN', 'f13', 'f11', 'f12', 'f10', 'f9', 'f15', 'f14', 'orion-IRc2', 'Orion_KL', 'OMC1_SE', 'BN-KL', 'OMC1_NW', 'f23', 'f16', 'OMC-1S', 'OrionKL-SV', 'OMC-1', 'OrionBullets', 'ONC', 'Orion_BNKL_source_I', 'OMC-1_Region5', 'OMC-1_Region1', 'Orion', 'OMC-1_Region2', 'HC602_HC606_HC608', 'GEMS28', 'HC672', 'ONC_Mosaic', 'OMC-1_Region3', 'OMC-1_Region4', 'Orion_KL_Field_1_Orion_Hot_Core', 'Orion_KL_Field_2_SMA1', 'Orion_KL_Field_3_North-west_Clump', 'Orion KL', 'Orion1', '101', '32', '104', '107', '71', 'Orion-KL', 'ORS-8', 'ORS-4']
--------------------------------
Target = AB Aur
--------------------------------
Number o

## 1.2 Query by position

The [<code>alminer.conesearch</code>](../pages/api.rst#alminer.conesearch) and [<code>alminer.catalog</code>](../pages/api.rst#alminer.catalog) functions can be used to directly query the ALMA archive by positions in the sky and a search radius around them. The right ascension and declinations must be given in units of degrees (ICRS). You can use the [Astropy coordinates package](https://docs.astropy.org/en/stable/coordinates/index.html) to convert your desired coordinates to degrees.

### Example 1.2.1:  query an object by its coordinates (RA, Dec)

In [6]:
myquery = alminer.conesearch(ra=201.365063, dec=-43.019112, point=False, search_radius=10.0)

--------------------------------
Number of projects = 24
Number of observations = 77
Number of unique subbands = 192
Total number of subbands = 312
9 target(s) with ALMA data = ['J1325-430', 'Centaurus_A', 'J1325-4301', 'CenA', 'Centaurus_a', '3FGL_J1325.4-4301', 'Centaurus A', 'NGC_5128', 'Cen_A']
--------------------------------


### Example 1.2.2:  query a catalog of objects by their coordinates (RA, Dec)

Let's first import a catalog, for example the catalog of Spitzer YSOs in Orion from Megeath et al. (2009), and create a PANDAS DataFrame using rows 866 to 869 of this catalog. Then use the [<code>alminer.catalog</code>](../pages/api.rst#alminer.catalog) function to query the ALMA archive for each target in the DataFrame.

In [7]:
Spitzer = ascii.read("Spitzer_sample.dat", header_start=0, data_start=866, data_end=869)

mycat =  {"Name": Spitzer["Seq"], 
          "RAJ2000" : Spitzer["RA2000"], 
          "DEJ2000" : Spitzer["DEC2000"]}

mycat = pandas.DataFrame(mycat)

myquery = alminer.catalog(mycat)

alminer.catalog results
Target = 866
--------------------------------
Number of projects = 1
Number of observations = 1
Number of unique subbands = 4
Total number of subbands = 4
1 target(s) with ALMA data = ['M12_866']
--------------------------------
Target = 867
--------------------------------
Number of projects = 1
Number of observations = 1
Number of unique subbands = 4
Total number of subbands = 4
1 target(s) with ALMA data = ['M12_867']
--------------------------------
Target = 868
--------------------------------
Number of projects = 2
Number of observations = 2
Number of unique subbands = 8
Total number of subbands = 8
1 target(s) with ALMA data = ['HOPS-172']
--------------------------------


## 1.3 Query by ALMA keywords

Query the ALMA archive for any [(string-type) keywords defined in ALMA TAP system](../pages/query_keywords.rst) using the [<code>alminer.keysearch</code>](../pages/api.rst#alminer.keysearch) function.

The power of this function is in combining keywords. When multiple keywords are provided, they are queried using 'AND' logic, but when multiple values are provided for a given keyword, they are queried using 'OR' logic. If a given value contains spaces, its constituents are queried using 'AND' logic. Words encapsulated in quotation marks (either ' or ") are queried as phrases. For example,

 * `alminer.keysearch({"proposal_abstract": ["high-mass star formation outflow disk"]})`
will query the archive for projects with the words "high-mass" AND "star" AND "formation" AND "outflow" AND "disk" in their proposal abstracts.
<br>

 * `alminer.keysearch({"proposal_abstract": ["high-mass", "star", "formation", "outflow", "disk"]})`
will query the archive for projects with the words "high-mass" OR "star" OR "formation" OR "outflow" OR "disk" in their proposal abstracts.
<br>

 * `alminer.keysearch({"proposal_abstract": ["'high-mass star formation' outflow disk"]})`
 will query the archive for projects with the phrase "high-mass star formation" AND the words "outflow" AND "disk" in their proposal abstracts.
<br>

 * `alminer.keysearch({"proposal_abstract": ["star formation"], "scientific_category":['Galaxy evolution']})`
will query the archive for projects with the words "star" AND "formation" in their proposal abstracts AND projects that are within the scientific_category of 'Galaxy evolution'.


<div class="alert alert-info">

Note

* Tables of ALMA [scientific categories](../pages/scientific_categories.rst) and [science keywords](../pages/science_keywords.rst) are provided on the sidebar. 

* For an overview, see [Appendix D of the ALMA Proposer’s
Guide](https://almascience.eso.org/proposing/proposers-guide#section-63).

</div>

### Example 1.3.1: query a list of ALMA target names that may not be in SIMBAD/NED/VizieR

In [8]:
myquery = alminer.keysearch({'target_name': ['GRB021004','SPT0319-47', 'G345']})

alminer.keysearch results 
--------------------------------
Number of projects = 17
Number of observations = 31
Number of unique subbands = 121
Total number of subbands = 191
12 target(s) with ALMA data = ['GRB021004', 'G345.5', 'SPT0319-47', 'G345.5043+00.3480', 'G345.49+1.47', 'G345.50+0.35', 'G345.6487+0.0089', 'G345.01', 'G345.11', 'G345.0029-0.2241', 'G345.5+1.5', 'G345.144-00.216']
--------------------------------


### Example 1.3.2: query a list of ALMA projects by their proposal IDs

In [9]:
myquery = alminer.keysearch({'proposal_id': ['2015.1.00664.S', '2016.1.00204.S']})

alminer.keysearch results 
--------------------------------
Number of projects = 2
Number of observations = 16
Number of unique subbands = 16
Total number of subbands = 64
16 target(s) with ALMA data = ['KMOS3DCOS4-24763', 'KMOS3DGS4-25151', 'KMOS3DCOS4-13701', 'KMOS3DCOS4-10347', 'KMOS3DCOS4-13174', 'KMOS3DCOS4-19680', 'KMOS3DCOS4-15813', 'KMOS3DCOS4-15820', 'KMOS3DU4-34138', 'KMOS3DU4-22227', 'KMOS3DU4-32147', 'KMOS3DU4-20547', 'KMOS3DGS4-11016', 'KMOS3DGS4-24110', 'KMOS3DGS4-27882', 'AK_Sco']
--------------------------------


### Example 1.3.3: query by words in the proposal abstract

Query the ALMA archive for proposals that have the phrase 'high-mass star formation' AND the words 'outflow' AND 'disk', OR the phrase 'massive star formation' AND the words 'outflow' AND 'disk' - and do not print the long list of target names in the summary.

In [10]:
myquery = alminer.keysearch({'proposal_abstract': ['"high-mass star formation" outflow disk', 
                                                   '"massive star formation" outflow disk']}, 
                            print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 14
Number of observations = 59
Number of unique subbands = 206
Total number of subbands = 423
Total number of targets with ALMA data = 29
--------------------------------


### Example 1.3.4: query by combination of keywords

Query the ALMA archive for proposals that have the phrase 'star formation' in their abstracts and correspond to the scientific category of 'Galaxy evolution'.

In [11]:
myquery = alminer.keysearch({'proposal_abstract': ['"star formation"'], 
                             'scientific_category':['Galaxy evolution']}, print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 245
Number of observations = 2839
Number of unique subbands = 3908
Total number of subbands = 11687
Total number of targets with ALMA data = 1763
--------------------------------


### Example 1.3.5: query for full polarization data

In [ ]:
myquery = alminer.keysearch({'science_keyword':['"disks around low-mass stars"'], 
                             'pol_states':['XY', 'YX']}, print_targets=False)

alminer.keysearch results 
